In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
# paths to files
full_data_file = '/share/garg/311_data/sb2377/clean_codebase/full.h5'
base_data_file = '/share/garg/311_data/sb2377/clean_codebase/three_year_base.csv'

# define covariates
normalized_covariates = ['normalized_log_population_density',
                         'normalized_log_income_median',
                         'normalized_education_bachelors_pct',
                         'normalized_race_white_nh_pct',
                         'normalized_age_median',
                         'normalized_households_renteroccupied_pct',
                         'normalized_rating']
thetas = ['theta.{}'.format(i) for i in range(1, len(normalized_covariates))]

In [3]:
# load files
data_df = pd.read_hdf(full_data_file, 'df')
base_df = pd.read_csv(base_data_file)

In [4]:
# formula to create synthetic ratings
# r_{ikt} = (1 / alpha_k) * (logit(E_t[T_{ikt}]) - theta_k X_i)

# in this notebook we will generate the synthetic coefficients theta_k and alpha_k (except the intercept)
# To reflect real-world reporting parameters, theta_k is drawn from N(mu, 0.1)
# mu = average reporting coefficients predicted by a logistic regression model run on the real inspection rating data.

# alpha_k is set to 1 so that the generated ratings have a standard deviation of 1

# the intercept (theta_k[0]) is set such that the generated ratings are mean 0 (see semisynthetic_intercept.py)

In [5]:
# get mu = average coefficients
# get data with observed ratings
observed_df = data_df[data_df['real_rating_observed'] == 1]

# get type indices for types with observed ratings
type_df = base_df[['typeagency', 'type_idxs']].drop_duplicates()
street_idx = type_df[type_df['typeagency'] == 'StreetConditionDOT']['type_idxs'].iloc[0]
park_idx = type_df[type_df['typeagency'] == 'MaintenanceorFacilityDPR']['type_idxs'].iloc[0]
rodent_idx = type_df[type_df['typeagency'] == 'RodentDOHMH']['type_idxs'].iloc[0]
restaurant_idx = type_df[type_df['typeagency'] == 'FoodDOHMH']['type_idxs'].iloc[0]
dcwp_idx = type_df[type_df['typeagency'] == 'ConsumerComplaintDCWP']['type_idxs'].iloc[0]

# get type specific data with observed ratings
street_observed_df = observed_df[observed_df['type_idxs'] == street_idx]
park_observed_df = observed_df[observed_df['type_idxs'] == park_idx]
rodent_observed_df = observed_df[observed_df['type_idxs'] == rodent_idx]
restaurant_observed_df = observed_df[observed_df['type_idxs'] == restaurant_idx]
dcwp_observed_df = observed_df[observed_df['type_idxs'] == dcwp_idx]

# fit logreg for each type separately
formula = 'finegrained_reported ~ '
for c in normalized_covariates:
    formula += c + ' + '
formula = formula[:-3]
street_fit = sm.Logit.from_formula(formula, data = street_observed_df).fit()
park_fit = sm.Logit.from_formula(formula, data = park_observed_df).fit()
rodent_fit = sm.Logit.from_formula(formula, data = rodent_observed_df).fit()
restaurant_fit = sm.Logit.from_formula(formula, data = restaurant_observed_df).fit()
dcwp_fit = sm.Logit.from_formula(formula, data = dcwp_observed_df).fit()

street_coefficients = np.expand_dims(street_fit.params.to_numpy(), axis=1)
park_coefficients = np.expand_dims(park_fit.params.to_numpy(), axis=1)
rodent_coefficients = np.expand_dims(rodent_fit.params.to_numpy(), axis=1)
restaurant_coefficients = np.expand_dims(restaurant_fit.params.to_numpy(), axis=1)
dcwp_coefficients = np.expand_dims(dcwp_fit.params.to_numpy(), axis=1)
all_coefficients = np.concatenate([street_coefficients, park_coefficients, rodent_coefficients, restaurant_coefficients, dcwp_coefficients], axis=1)

# mu = average across types with observed ratings
mean_coefficients = all_coefficients.mean(axis=1)

Optimization terminated successfully.
         Current function value: 0.028475
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.158972
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.013385
         Iterations 12
Optimization terminated successfully.
         Current function value: 0.045373
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.094803
         Iterations 8


In [ ]:
# type specific theta_k and alpha_k are drawn from N(mu, 0.1)
num_sets = 100
mean_coefficients[-1] = -1 # set to -1 so that generated ratings have a standard deviation ~= 1 
error_coefficients = np.random.normal(loc=0, scale=0.1, size=(num_sets, len(type_df), len(mean_coefficients)))
type_specific_coefficients = mean_coefficients + error_coefficients

# create a coefficient df
# alpha = theta.(len(normalized_covariates))
coeff_df = pd.DataFrame()
coeff_df['type_idxs'] = np.arange(type_specific_coefficients.shape[1])
for j in range(num_sets):
    for i in range(1, type_specific_coefficients.shape[2]):
        coeff_df['theta.{}_{}'.format(i, j)] = type_specific_coefficients[j, :, i]
        
# save mu = mean coefficients
for i in range(1, type_specific_coefficients.shape[2]):
    coeff_df['mean_theta.{}'.format(i)] = mean_coefficients[i]

In [10]:
# as mentioned earlier, the intercept (theta_k[0]) is set such that the generated ratings are mean 0 
# thus, theta_k[0] = logit(E_t[T_{ikt}]) - theta_k[1:] X_i[1:]

# in order to generate the intercepts we need
# (i) synthetic values for theta_k[1:]
# (ii) an estimate of E_t[T_{ikt}]

# we have already generated (i) above
# we now estimate E_t[T_{ikt}], or the empirical reporting frequencies, across the full dataset (train + test) 

In [11]:
# get empirical E_t[T_{ikt}] or P(T) estimates from full dataset
# E_t[T_{ikt}] is the mean value of T across time (t) for each node, type pair (i,k)
empirical_pt = data_df.groupby(['type_idxs', 'node_idxs'])['finegrained_reported'].mean()
empirical_pt_df = empirical_pt.reset_index()
empirical_pt_df = empirical_pt_df.rename(columns={'finegrained_reported': 'P(T)'})
# map to full dataset
data_df = pd.merge(data_df, empirical_pt_df, on=['type_idxs', 'node_idxs'])

# get number of data points for each node/type pair
num_entries = data_df.groupby(['type_idxs', 'node_idxs']).size().reset_index()
num_entries = num_entries.rename(columns={0: 'num_rows'})
data_df = pd.merge(data_df, num_entries, on=['type_idxs', 'node_idxs'])

In [12]:
# to make semisynthetic data well specified, we do not want any node/type pairs where P(T) = 0 or P(T) = 1
# thus we randomly bitflip one datapoint for these sets
# first we define helper functions

# Group by 'node' and 'type', and apply a lambda to set one random 'T' to 1 per group
def set_random_one(group):
    # Choosing a random index from the group's indices
    random_index = np.random.choice(group.index)
    # Setting 'T' to 1 at the chosen index
    group.at[random_index, 'bitflip_reported'] = 1
    return group

# Group by 'node' and 'type', and apply a lambda to set one random 'T' to 0 per group
def set_random_zero(group):
    # Choosing a random index from the group's indices
    random_index = np.random.choice(group.index)
    # Setting 'T' to 1 at the chosen index
    group.at[random_index, 'bitflip_reported'] = 0
    return group

In [13]:
# bitflip node/type pairs with P(T) = 0
pt0_df = data_df[data_df['P(T)'] == 0].copy()
pt0_df['bitflip_reported'] = pt0_df['finegrained_reported']

# Apply the function across the DataFrame grouped by 'node' and 'type'
pt0_df = pt0_df.groupby(['node_idxs', 'type_idxs']).apply(set_random_one).reset_index(drop=True)
pt0_df['bitflip_P(T)'] = 1 / pt0_df['num_rows']

# bitflip node/type pairs with P(T) = 1
pt1_df = data_df[data_df['P(T)'] == 1].copy()
pt1_df['bitflip_reported'] = pt1_df['finegrained_reported']

# Apply the function across the DataFrame grouped by 'node' and 'type'
pt1_df = pt1_df.groupby(['node_idxs', 'type_idxs']).apply(set_random_zero).reset_index(drop=True)
pt1_df['bitflip_P(T)'] = (pt1_df['num_rows'] - 1) / pt1_df['num_rows']

# add in bitflipped information
no_bitflip = data_df[(data_df['P(T)'] != 0) & (data_df['P(T)'] != 1)].copy()
no_bitflip['bitflip_reported'] = no_bitflip['finegrained_reported']
no_bitflip['bitflip_P(T)'] = no_bitflip['P(T)']
data_df = pd.concat([no_bitflip, pt0_df, pt1_df])

In [14]:
assert(len(coeff_df[coeff_df.isna().any(axis=1)]) == 0)
assert(len(data_df[data_df.isna().any(axis=1)]) == 0)

In [16]:
coeff_df.to_csv('/share/garg/311_data/sb2377/clean_codebase/semisynthetic/semisynthetic_coeffs.csv')

In [17]:
data_df.to_hdf('/share/garg/311_data/sb2377/clean_codebase/semisynthetic/semisynthetic_full.h5', key='df', mode='w')

/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_972253/2408036819.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['typeagency', 'finegrained_id'], dtype='object')]

  data_df.to_hdf('/share/garg/311_data/sb2377/clean_codebase/semisynthetic/semisynthetic_full.h5', key='df', mode='w')
